In [3]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import random
from itertools import combinations
from dataclasses import dataclass
from typing import List, Dict, Set, Tuple
import math
from collections import defaultdict

# Constants
POSITIONS = ['top', 'jgl', 'mid', 'bot', 'sup']
RANKS = ['iron', 'bronze', 'silver', 'gold', 'platinum', 'emerald', 'diamond', 'master', 'grandmaster', 'challenger']
SERVERS = ['korea', 'china', 'others']
HIGH_SKILL_SERVERS = {'korea', 'china'}

#####

@dataclass
class PositionPreference:
    main_roles: List[str]
    secondary_roles: List[str]
    tertiary_roles: List[str]

    def __post_init__(self):
        # Validate that each position appears only once
        all_positions = self.main_roles + self.secondary_roles + self.tertiary_roles
        if len(all_positions) != len(set(all_positions)):
            raise ValueError("A position cannot appear in multiple tiers")
        
        # Validate that all positions are valid
        for pos in all_positions:
            if pos not in POSITIONS:
                raise ValueError(f"Invalid position: {pos}")

    def get_all_positions(self) -> List[str]:
        """Return all positions the player can play"""
        return list(set(self.main_roles + self.secondary_roles + self.tertiary_roles))

    def get_position_tier(self, position: str) -> int:
        """
        Returns the tier of the position (1 for main, 2 for secondary, 3 for tertiary)
        Returns -1 if position is not in any tier
        """
        if position in self.main_roles:
            return 1
        elif position in self.secondary_roles:
            return 2
        elif position in self.tertiary_roles:
            return 3
        return -1

    def __str__(self):
        return (f"Main: {', '.join(self.main_roles) if self.main_roles else 'None'} | "
                f"Secondary: {', '.join(self.secondary_roles) if self.secondary_roles else 'None'} | "
                f"Tertiary: {', '.join(self.tertiary_roles) if self.tertiary_roles else 'None'}")

#####
    
@dataclass
class Player:
    name: str
    rank: str
    position_preference: PositionPreference
    server: str
    order_capable: bool = False

    @property
    def rank_points(self) -> float:
        base_points = RANKS.index(self.rank.lower()) + 1
        server_bonus = 1 if self.server.lower() in HIGH_SKILL_SERVERS else 0
        return base_points + server_bonus

    def position_penalty(self, assigned_position: str) -> float:
        tier = self.position_preference.get_position_tier(assigned_position)
        if tier == 1:  # Main role
            return 0
        elif tier == 2:  # Secondary role
            return -0.5
        elif tier == 3:  # Tertiary role
            return -1.0
        return float('-inf')  # Position not in any tier

    @property
    def order_points(self) -> float:
        return 0.5 if self.order_capable else 0

    @property
    def position_flexibility(self) -> int:
        """Return the total number of positions the player can play"""
        return len(self.position_preference.get_all_positions())

#####

class PlayerCreationFrame(ttk.Frame):
    def __init__(self, parent, add_player_callback):
        super().__init__(parent)
        self.add_player_callback = add_player_callback
        self.position_vars = {
            'main': [],
            'secondary': [],
            'tertiary': []
        }
        self.setup_ui()

    def setup_ui(self):
        # Player Name
        ttk.Label(self, text="Player Name:").grid(row=0, column=0, padx=5, pady=5)
        self.name_entry = ttk.Entry(self)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)

        # Rank Selection
        ttk.Label(self, text="Rank:").grid(row=1, column=0, padx=5, pady=5)
        self.rank_var = tk.StringVar()
        rank_combo = ttk.Combobox(self, textvariable=self.rank_var, values=RANKS)
        rank_combo.grid(row=1, column=1, padx=5, pady=5)
        rank_combo.set(RANKS[0])

        # Server Selection
        ttk.Label(self, text="Server:").grid(row=2, column=0, padx=5, pady=5)
        self.server_var = tk.StringVar()
        server_combo = ttk.Combobox(self, textvariable=self.server_var, values=SERVERS)
        server_combo.grid(row=2, column=1, padx=5, pady=5)
        server_combo.set(SERVERS[0])

        # Order Capability
        ttk.Label(self, text="Order Capable:").grid(row=3, column=0, padx=5, pady=5)
        self.order_var = tk.BooleanVar()
        ttk.Checkbutton(self, variable=self.order_var).grid(row=3, column=1, padx=5, pady=5)

        # Position Preferences
        self.create_position_section("Main Roles", 4, 'main')
        self.create_position_section("Secondary Roles", 5, 'secondary')
        self.create_position_section("Tertiary Roles", 6, 'tertiary')

        # Add Player Button
        ttk.Button(self, text="Add Player", command=self.add_player).grid(row=7, column=0, columnspan=2, pady=20)

    def create_position_section(self, title: str, row: int, role_type: str):
        ttk.Label(self, text=f"{title}:").grid(row=row, column=0, padx=5, pady=5)
        position_frame = ttk.Frame(self)
        position_frame.grid(row=row, column=1, padx=5, pady=5)

        for i, pos in enumerate(POSITIONS):
            var = tk.BooleanVar()
            self.position_vars[role_type].append((pos, var))
            ttk.Checkbutton(position_frame, text=pos, variable=var).grid(row=0, column=i, padx=2)

    def get_selected_positions(self, role_type: str) -> List[str]:
        return [pos for pos, var in self.position_vars[role_type] if var.get()]

    def add_player(self):
        try:
            # Get position preferences
            positions = {
                'main': self.get_selected_positions('main'),
                'secondary': self.get_selected_positions('secondary'),
                'tertiary': self.get_selected_positions('tertiary')
            }

            # Validate that a position is not selected in multiple tiers
            all_positions = positions['main'] + positions['secondary'] + positions['tertiary']
            if len(all_positions) != len(set(all_positions)):
                raise ValueError("A position cannot be selected in multiple tiers")

            # Validate that at least one main role is selected
            if not positions['main']:
                raise ValueError("At least one main role must be selected")

            # Create player
            player = Player(
                name=self.name_entry.get().strip(),
                rank=self.rank_var.get(),
                position_preference=create_position_preference(positions),
                server=self.server_var.get(),
                order_capable=self.order_var.get()
            )

            self.add_player_callback(player)
            self.clear_form()
            messagebox.showinfo("Success", "Player added successfully!")

        except ValueError as e:
            messagebox.showerror("Error", str(e))

    def clear_form(self):
        self.name_entry.delete(0, tk.END)
        self.rank_var.set(RANKS[0])
        self.server_var.set(SERVERS[0])
        self.order_var.set(False)
        for role_type in self.position_vars:
            for _, var in self.position_vars[role_type]:
                var.set(False)

#####
                
class PlayerListFrame(ttk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.players: List[Player] = []
        self.setup_ui()

    def setup_ui(self):
        # Player List
        self.tree = ttk.Treeview(self, columns=('Name', 'Rank', 'Server', 'Positions', 'Order'), show='headings')
        self.tree.heading('Name', text='Name')
        self.tree.heading('Rank', text='Rank')
        self.tree.heading('Server', text='Server')
        self.tree.heading('Positions', text='Positions')
        self.tree.heading('Order', text='Order Capable')
        
        scrollbar = ttk.Scrollbar(self, orient=tk.VERTICAL, command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)

        self.tree.grid(row=0, column=0, sticky='nsew')
        scrollbar.grid(row=0, column=1, sticky='ns')

        # Generate Teams Button
        ttk.Button(self, text="Generate Teams", command=self.generate_teams).grid(row=1, column=0, pady=10)

        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)

    def add_player(self, player: Player):
        self.players.append(player)
        positions = str(player.position_preference)
        self.tree.insert('', 'end', values=(
            player.name,
            player.rank,
            player.server,
            positions,
            'Yes' if player.order_capable else 'No'
        ))

    def generate_teams(self):
        if len(self.players) < 10:
            messagebox.showwarning("Warning", f"Not enough players! Have {len(self.players)}/10 players.")
            return

        # Create results window
        results_window = tk.Toplevel()
        results_window.title("Team Compositions")
        results_window.geometry("1000x800")

        # Create text widget for results
        text_widget = tk.Text(results_window, wrap=tk.WORD, padx=10, pady=10)
        scrollbar = ttk.Scrollbar(results_window, orient=tk.VERTICAL, command=text_widget.yview)
        text_widget.configure(yscrollcommand=scrollbar.set)

        text_widget.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        # Generate and display team compositions
        top_compositions = generate_team_combinations(self.players, num_combinations=5)
        
        for i, comp in enumerate(top_compositions, 1):
            text_widget.insert(tk.END, f"\nTeam Composition #{i}\n")
            text_widget.insert(tk.END, "=" * 80 + "\n")
            text_widget.insert(tk.END, str(comp) + "\n")
            text_widget.insert(tk.END, "=" * 80 + "\n\n")

        # Add button to save results
        save_button = ttk.Button(results_window, text="Save Results", 
                               command=lambda: self.save_results(text_widget.get("1.0", tk.END)))
        save_button.pack(pady=10)

    def save_results(self, results: str):
        """Save the team composition results to a file"""
        try:
            with open("team_compositions.txt", "w") as f:
                f.write(results)
            messagebox.showinfo("Success", "Results saved to team_compositions.txt")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save results: {str(e)}")
            
#####

@dataclass
class TeamComposition:
    red_team: Dict[str, Player]
    blue_team: Dict[str, Player]
    red_score: float
    blue_score: float
    t_value: float  # Total score difference
    l_value: float  # Lane difference RMS

    def __str__(self):
        result = []
        # Red Team
        result.append("RED TEAM")
        result.append(f"Team Score: {self.red_score:.2f}")
        result.append("-" * 50)
        for position in POSITIONS:
            player = self.red_team[position]
            result.append(f"{position.upper()}: {player.name} ({player.rank}, {player.server})")
            result.append(f"  Position Preference: {player.position_preference}")
            result.append(f"  Order Capable: {player.order_capable}")
        
        # Blue Team
        result.append("\nBLUE TEAM")
        result.append(f"Team Score: {self.blue_score:.2f}")
        result.append("-" * 50)
        for position in POSITIONS:
            player = self.blue_team[position]
            result.append(f"{position.upper()}: {player.name} ({player.rank}, {player.server})")
            result.append(f"  Position Preference: {player.position_preference}")
            result.append(f"  Order Capable: {player.order_capable}")
        
        # Metrics
        result.append("\nMETRICS")
        result.append(f"T-Value (Team Score Difference): {self.t_value:.2f}")
        result.append(f"L-Value (Lane Difference RMS): {self.l_value:.2f}")
        
        return "\n".join(result)

def calculate_t_value(red_score: float, blue_score: float) -> float:
    """Calculate the absolute difference between team scores"""
    return abs(red_score - blue_score)

def calculate_l_value(red_team: Dict[str, Player], blue_team: Dict[str, Player]) -> float:
    """Calculate the root mean square of lane differences"""
    squared_diffs = []
    
    for position in POSITIONS:
        red_player = red_team[position]
        blue_player = blue_team[position]
        
        # Calculate individual position score difference
        red_pos_score = (red_player.rank_points + 
                        red_player.position_penalty(position) + 
                        red_player.order_points)
        blue_pos_score = (blue_player.rank_points + 
                         blue_player.position_penalty(position) + 
                         blue_player.order_points)
        
        diff = red_pos_score - blue_pos_score
        squared_diffs.append(diff ** 2)
    
    # Calculate RMS
    mean_squared_diff = sum(squared_diffs) / len(POSITIONS)
    return math.sqrt(mean_squared_diff)

def evaluate_team_composition(players: List[Player], position_assignments: Dict[str, Player]) -> float:
    """Evaluate a team composition based on various metrics"""
    total_score = 0
    
    for position, player in position_assignments.items():
        # Add rank points
        total_score += player.rank_points
        
        # Add/subtract points based on position preference
        total_score += player.position_penalty(position)
        
        # Add points for order capability
        total_score += player.order_points
    
    # Add team synergy bonus based on average rank
    avg_rank = sum(p.rank_points for p in players) / len(players)
    total_score += math.log(avg_rank)
    
    return total_score

def generate_team_combinations(players: List[Player], num_combinations: int = 5) -> List[TeamComposition]:
    """Generate and evaluate possible team compositions, returning the top combinations"""
    valid_combinations = []
    
    # Generate all possible 10-player combinations divided into two teams
    for team_players in combinations(players, 10):
        # Try different ways to split players into two teams
        for red_team_players in combinations(team_players, 5):
            blue_team_players = tuple(p for p in team_players if p not in red_team_players)
            
            # Generate valid position assignments for both teams
            red_assignments = generate_valid_position_assignments(list(red_team_players))
            blue_assignments = generate_valid_position_assignments(list(blue_team_players))
            
            # Try all valid position combinations
            for red_pos in red_assignments:
                for blue_pos in blue_assignments:
                    # Evaluate both teams
                    red_score = evaluate_team_composition(list(red_team_players), red_pos)
                    blue_score = evaluate_team_composition(list(blue_team_players), blue_pos)
                    
                    # Calculate metrics
                    t_value = calculate_t_value(red_score, blue_score)
                    l_value = calculate_l_value(red_pos, blue_pos)
                    
                    comp = TeamComposition(
                        red_team=red_pos,
                        blue_team=blue_pos,
                        red_score=red_score,
                        blue_score=blue_score,
                        t_value=t_value,
                        l_value=l_value
                    )
                    valid_combinations.append(comp)
    
    # Sort by balance (lower T and L values are better)
    return sorted(valid_combinations, 
                 key=lambda x: (x.t_value + x.l_value))[:num_combinations]

#####

class MainApplication(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Team Composition Generator")
        self.geometry("1200x800")

        # Create main container
        container = ttk.Frame(self)
        container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create and arrange the two main frames
        self.player_list = PlayerListFrame(container)
        self.player_creation = PlayerCreationFrame(container, self.player_list.add_player)

        self.player_creation.grid(row=0, column=0, padx=10, pady=10, sticky='nsew')
        self.player_list.grid(row=0, column=1, padx=10, pady=10, sticky='nsew')

        # Configure grid weights
        container.grid_columnconfigure(1, weight=1)
        container.grid_rowconfigure(0, weight=1)

def run_gui():
    app = MainApplication()
    app.mainloop()

if __name__ == "__main__":
    run_gui()